In [3]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))
sys.path.append(project_path)
print(f"Project Path: {project_path}")

Project Path: c:\Users\dalej\Documents\_Coding\DragonRegen


## Testing Kafka Agent

In [4]:
from src.AIGuardian.Tasks.KafkaAgent import KafkaAgent

agent = KafkaAgent(end_processing=20)
agent.receive_messages()

Agent Loop Count: 0 and last consumed time: 2025-05-22 17:31:46.905922
Agent Loop Count: 1 and last consumed time: 2025-05-22 17:31:51.107552
Agent Loop Count: 2 and last consumed time: 2025-05-22 17:31:55.126427
==> Running DataColumnType with prompt: None
==> Task dd3dc272-6a17-46cb-a76e-48a237ff4529 complete_task
==> Running DataColumnType with prompt: None
==> Task 7bc36d4b-a4e0-4b62-9cf8-efafadcdd132 complete_task
==> Running DataColumnType with prompt: None
==> Task b4efeef3-3456-4d25-b321-c7a039602b05 complete_task
==> Removing dd3dc272-6a17-46cb-a76e-48a237ff4529 from child tasks.
==> Removing 3927bf8d-7687-44da-90f1-42d7dbd2244d from child tasks.
Agent Loop Count: 3 and last consumed time: 2025-05-22 17:32:05.821649
==> Running DataColumnRefiner with prompt: None
==> Task eae5db2f-82fb-4e4b-b994-2cfdb6c16cea complete_task
==> Running DataColumnRefiner with prompt: None
==> Task 19aa66fa-0a64-46ec-b0c6-857fe9caf755 complete_task
==> Running DataColumnRefiner with prompt: None
=

In [29]:
print(agent.waiting_tasks)
print(agent.processed_tasks)
print(agent.start_time)
print(agent.end_time)

[]
[<src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x0000027E45E13C90>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x0000027E45E13C50>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x0000027E45E13FD0>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x0000027E45E18310>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x0000027E45EE82D0>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x0000027E45EEB910>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x0000027E45E02450>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x0000027E45F05390>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x0000027E45F1CD50>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x0000027E45F04DD0>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x0000027E45DF7D50>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000

In [ ]:
if agent.waiting_tasks:
    completed_tasks = agent.db_engine.consumers[AILoggingTopics.AI_TASK_COMPLETED_TOPIC].poll(timeout_ms=1000, max_records=20)
    tasks_to_remove = []
    for task in agent.waiting_tasks:
        agent.completed_tasks_received_d1[task.task_id] = agent.completed_tasks_received_d1.get(task.task_id, [])  + agent.kafka_records_to_list(completed_tasks, 'task_id')
        task.wait_on_dependency_check(completed_tasks)
        if not task.is_waiting():
            print("BOOOOOMM!")
            task.complete_task()
            agent.completed_tasks.append(task)
            tasks_to_remove.append(task)
    # Remove the completed task after interating through it
    for r_task in tasks_to_remove:
        agent.waiting_tasks.remove(r_task)
    agent.completed_tasks_received = agent.completed_tasks_received + agent.kafka_records_to_list(completed_tasks, 'task_id')

==> Removing 629fc112-6e53-482b-a403-79e66f5ba9ff from child tasks.
BOOOOOMM!
====> COLUMN REFINER XXXCC: Complete Task 5230a66d-9f80-448d-9cf6-4559f95bc94c
 Adding Key: 00e02a24-9cd1-4f0c-b923-b0986b7b4bdb to the list of fields
 Adding Key: 722d81b5-1402-46a4-9a9b-2a0e93e4c21e to the list of fields
 Adding Key: 5bf50e77-380f-420c-9227-e84b34bdff36 to the list of fields
 Adding Key: 629fc112-6e53-482b-a403-79e66f5ba9ff to the list of fields
==> Task 5230a66d-9f80-448d-9cf6-4559f95bc94c complete_task


NameError: name 'datetime' is not defined

In [28]:
for r_task in tasks_to_remove:
    agent.waiting_tasks.remove(r_task)
agent.completed_tasks_received = agent.completed_tasks_received + agent.kafka_records_to_list(completed_tasks, 'task_id')

In [18]:
from src.MetaFort.AILoggingTopics import AILoggingTopics
print(agent.waiting_tasks[0].child_task)
print(agent.db_engine.consumers[AILoggingTopics.AI_TASK_COMPLETED_TOPIC].config['group_id'])
print(agent.waiting_tasks[0].is_waiting())


['629fc112-6e53-482b-a403-79e66f5ba9ff']
task-completed-fd639567-e496-48c4-a9db-9b0529dd6f14
True


---
Looking into issues with kafka missing waits

In [19]:
ls_waiting_tasks = []
ls_waiting_tasks_child = {}
for item in agent.waiting_tasks:
    if item.is_waiting():
        ls_waiting_tasks.append(item.task_id)
        ls_waiting_tasks_child[item.task_id] = item.child_task
print(len(ls_waiting_tasks))
print(len(ls_waiting_tasks_child))


1
1


In [20]:
ls_received_tasks = []
for item in agent.received_tasks:
    ls_received_tasks.append(item.task_id)
print(len(ls_received_tasks))

63


In [21]:
ls_processed_tasks = []
for item in agent.processed_tasks:
    ls_processed_tasks.append(item.task_id)
print(len(ls_processed_tasks))

63


In [22]:
ls_completed_tasks = []
for item in agent.completed_tasks:
    ls_completed_tasks.append(item.task_id)
print(len(ls_completed_tasks))

62


In [24]:
print(ls_waiting_tasks_child)


{'5230a66d-9f80-448d-9cf6-4559f95bc94c': ['629fc112-6e53-482b-a403-79e66f5ba9ff']}


In [12]:
# search_task_id = 'f4f01633-eef2-4792-b531-64baae246232'
search_task_id = '16404101-100b-45dc-b8f5-e93eba5ee294'
if search_task_id in ls_processed_tasks:
    print("processed tasks")
if search_task_id in ls_received_tasks:
    print("received tasks")
if search_task_id in ls_completed_tasks:
    print("Task is in completed tasks")
if search_task_id in agent.completed_tasks_received:
    print("Task is in received completed tasks")

In [13]:
search_task_id = '78475bc0-9ed2-43e8-a652-fc7c480cf7af'
if search_task_id in ls_processed_tasks:
    print("processed tasks")
if search_task_id in ls_received_tasks:
    print("received tasks")
if search_task_id in ls_completed_tasks:
    print("Task is in completed tasks")